In [1]:
#### In this notebook we obtain the branching ratio for the processes zp -> chi chi, 
#### y0 -> chi chi, and h2 -> chi chi, and store the results in a dataframe that
#### contains the mediator mass, and the branching ratios

import sys,os,glob,copy
sys.path.append('../')
import numpy as np
from numpy.linalg import norm
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.interpolate import LinearNDInterpolator,interp2d
import scipy
import matplotlib as mpl
import sympy as sp
from matplotlib.colors import LogNorm



plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})

plt.rcParams.update({"savefig.dpi" : 300}) #Figure resolution


#Define plotting style:
sns.set() #Set style
sns.set_style('ticks',{'font.family':'Times New Roman', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
cm = plt.cm.get_cmap('RdYlBu')

## Objective
This notebook is dedicated to obtaining the numeric value for the branching ratio of the process Med $\to \chi \chi$, where Med is th particle that mediates the dark and visible sector, and $\chi$ is the dark matter particle. We consider two models: the two mediators dark matter (2MDM) model, and the simplified spin-1 dark matter model (DMSimp).

In [2]:
BRdata = pd.DataFrame(columns=['Model', 'Mediator', '$m_{med}$', '$m_{chi}$', '$g_{q}$', '$\sin\\alpha$', 
                               '$g_{\chi}$', '$y_{\chi}$', '$BR(med>\chi\chi)$'])

### 2MDM model

In [3]:
## spin 1 mediator ##

# mediator mass
MZp = np.arange(1000, 4050, 50)

# quarks masses
MU = 0.00255
MC = 1.27
MT = 172
MD = 0.00504
MS = 0.101
MB = 4.7

# DM mass
# Mchi = 65.0

# couplings
gqV = np.array([0.01, 0.1, 0.2, 0.25])
gchi = np.array([0.1, 0.5, 1.0, 1.5, 2.0])
gqA = 0.0
gZp = 1.0


In [4]:
for gq in gqV:
    for g in gchi:
        for m in MZp:
            for Mchi in np.arange(100., m/2, 50):
                gqL = (gqA + gq)
                gqR = (-gqA + gq)

                # Zp to t 
                if Mchi < 2*MT:
                    ZPtt = 0
                else:

                    ZPtt = ((-6*gqL**2*gZp**2*MT**2 + 36*gqL*gqR*gZp**2*MT**2 - 6*gqR**2*gZp**2*MT**2 
                             + 6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2)
                            *np.sqrt(-4*MT**2*m**2 + m**4))/(48.*np.pi*abs(m)**3)            

                # Zp to u
                ZPuu = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

                # Zp to c
                ZPcc = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

                # Zp to d
                ZPdd = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

                # Zp to s
                ZPss = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

                # Zp to b
                ZPbb = (m**2*(6*gqL**2*gZp**2*m**2 + 6*gqR**2*gZp**2*m**2))/(48.*np.pi*abs(m)**3)

                # Zp to DM
                ZPchichi = ((-16*g**2*Mchi**2 + 4*g**2*m**2)
                            *np.sqrt(-4*Mchi**2*m**2 + m**4))/(96.*np.pi*abs(m)**3)

                # Zp total width
                GammaZP = ZPuu + ZPcc + ZPtt + ZPdd + ZPss + ZPbb + ZPchichi

                # BR(Zp to DM)
                BR_ZPchichi = ZPchichi / GammaZP

                # BR(Zp to quarks)
                BR_ZPqq = (ZPuu + ZPcc + ZPtt + ZPdd + ZPss + ZPbb)/GammaZP

                BRdata.loc[len(BRdata)] = ['2MDM', 'spin-1', m, Mchi, gq, np.nan, g, np.nan, BR_ZPchichi]

In [5]:
BRdata

,Model,Mediator,$m_{med}$,$m_{chi}$,$g_{q}$,$\sin\alpha$,$g_{\chi}$,$y_{\chi}$,$BR(med>\chi\chi)$
0,2MDM,spin-1,1000,100.0,0.01,NaN,0.1,NaN,0.758182
1,2MDM,spin-1,1000,150.0,0.01,NaN,0.1,NaN,0.743169
2,2MDM,spin-1,1000,200.0,0.01,NaN,0.1,NaN,0.719593
3,2MDM,spin-1,1000,250.0,0.01,NaN,0.1,NaN,0.684051
4,2MDM,spin-1,1000,300.0,0.01,NaN,0.1,NaN,0.630542
...,...,...,...,...,...,...,...,...,...
28355,2MDM,spin-1,4000,1750.0,0.25,NaN,2.0,NaN,0.167858
28356,2MDM,spin-1,4000,1800.0,0.25,NaN,2.0,NaN,0.128339
28357,2MDM,spin-1,4000,1850.0,0.25,NaN,2.0,NaN,0.088859
28358,2MDM,spin-1,4000,1900.0,0.25,NaN,2.0,NaN,0.051345


In [7]:
## spin-0 mediator ##

# mediator mass
MSd = np.arange(500, 2050, 50)

# quarks masses
MT = 172

# tau lepton mass
MTA = 1.777

# DM mass
# Mchi = 65.0

# gauge bosons masses
MW = 79.824
MZ = 91.1876
MZp = 2000.0

# higgs boson mass
MH = 125.0

aEWM1 = 127.9
aEW = 1/aEWM1
ee = 2*np.sqrt(aEW)*np.sqrt(np.pi)

sw2 = 1 - MW**2/MZ**2
sw = np.sqrt(sw2)
cw = np.sqrt(1 - sw2)

# vevs
vev = (2*MW*sw)/ee
gchi = 1.0
vev2 = MZp/(2*gchi)

# couplings
ychi = np.array([0.1, 0.5, 1.0, 1.5, 2.0])
yt = np.sqrt(2)*MT/vev
ytau = np.sqrt(2)*MTA/vev
Sa = np.array([0.1, 0.2, 0.25])


In [9]:
for s in Sa:
    for y in ychi:
        for m in MSd:
            for Mchi in np.arange(100., m/2, 50):
                Ca = np.sqrt(1-s**2)
                lam1 = (Ca**2*MH**2)/(2.*vev**2) + (m**2*s**2)/(2.*vev**2)
                lam2 = (Ca**2*m**2)/(2.*vev2**2) + (MH**2*s**2)/(2.*vev2**2)
                lam3 = (Ca*(-MH**2 + m**2)*s)/(vev*vev2)

                # S to Higgs
                if m < 2* MH:
                    Shh = 0
                else:
                    Shh = ((36*Ca**4*lam1**2*s**2*vev**2 - 24*Ca**4*lam1*lam3*s**2*vev**2 + 4*Ca**4*lam3**2*s**2*vev**2 
                            + 12*Ca**2*lam1*lam3*s**4*vev**2 - 4*Ca**2*lam3**2*s**4*vev**2 + lam3**2*s**6*vev**2 
                            + 12*Ca**5*lam1*lam3*s*vev*vev2 - 4*Ca**5*lam3**2*s*vev*vev2 + 72*Ca**3*lam1*lam2*s**3*vev*vev2 
                            - 24*Ca**3*lam1*lam3*s**3*vev*vev2 - 24*Ca**3*lam2*lam3*s**3*vev*vev2 + 10*Ca**3*lam3**2*s**3*vev*vev2 
                            + 12*Ca*lam2*lam3*s**5*vev*vev2 - 4*Ca*lam3**2*s**5*vev*vev2 + Ca**6*lam3**2*vev2**2 
                            + 12*Ca**4*lam2*lam3*s**2*vev2**2 - 4*Ca**4*lam3**2*s**2*vev2**2 + 36*Ca**2*lam2**2*s**4*vev2**2 
                            - 24*Ca**2*lam2*lam3*s**4*vev2**2 + 4*Ca**2*lam3**2*s**4*vev2**2)
                           *np.sqrt(-4*MH**2*m**2 + m**4))/(32.*np.pi*abs(m)**3)

                # S to DM
                Schichi = ((-4*Ca**2*Mchi**2*y**2 + Ca**2*m**2*y**2)
                           *np.sqrt(-4*Mchi**2*m**2 + m**4))/(32.*np.pi*abs(m)**3)

                # S to tau
                Stautau = ((m**2*s**2*ytau**2 - 4*MTA**2*s**2*ytau**2)
                           *np.sqrt(m**4 - 4*m**2*MTA**2))/(16.*np.pi*abs(m)**3)

                # S to top
                if m < 2*MT:
                    Stt = 0
                else:
                    Stt = ((3*m**2*s**2*yt**2 - 12*MT**2*s**2*yt**2)
                           *np.sqrt(m**4 - 4*m**2*MT**2))/(16.*np.pi*abs(m)**3)

                # S to W
                if m < 2*MW:
                    Sww = 0
                else:
                    Sww = (((3*ee**4*s**2*vev**2)/(4.*sw**4) + (ee**4*m**4*s**2*vev**2)/(16.*MW**4*sw**4) 
                            - (ee**4*m**2*s**2*vev**2)/(4.*MW**2*sw**4))
                           *np.sqrt(m**4 - 4*m**2*MW**2))/(16.*np.pi*abs(m)**3)

                # S to Z
                if m < 2*MZ:
                    Szz = 0
                else:
                    Szz = (((9*ee**4*s**2*vev**2)/2. + (3*ee**4*m**4*s**2*vev**2)
                            /(8.*MZ**4) - (3*ee**4*m**2*s**2*vev**2)/(2.*MZ**2) 
                            + (3*cw**4*ee**4*s**2*vev**2)/(4.*sw**4) + (cw**4*ee**4*m**4*s**2*vev**2)
                            /(16.*MZ**4*sw**4) - (cw**4*ee**4*m**2*s**2*vev**2)
                            /(4.*MZ**2*sw**4) + (3*cw**2*ee**4*s**2*vev**2)/sw**2 
                            + (cw**2*ee**4*m**4*s**2*vev**2)/(4.*MZ**4*sw**2) 
                            - (cw**2*ee**4*m**2*s**2*vev**2)/(MZ**2*sw**2) + (3*ee**4*s**2*sw**2*vev**2)/cw**2 
                            + (ee**4*m**4*s**2*sw**2*vev**2)/(4.*cw**2*MZ**4) - (ee**4*m**2*s**2*sw**2*vev**2)/(cw**2*MZ**2) 
                            + (3*ee**4*s**2*sw**4*vev**2)/(4.*cw**4) + (ee**4*m**4*s**2*sw**4*vev**2)/(16.*cw**4*MZ**4) 
                            - (ee**4*m**2*s**2*sw**4*vev**2)/(4.*cw**4*MZ**2))
                           *np.sqrt(m**4 - 4*m**2*MZ**2))/(32.*np.pi*abs(m)**3)


                #  S Total width
                GammaS = Shh + Schichi + Stautau + Stt + Sww + Szz

                # BR(S to DM)
                BR_Schichi = Schichi / GammaS

                BRdata.loc[len(BRdata)] = ['2MDM', 'spin-0', m, Mchi, np.nan, s, np.nan, y, BR_Schichi]

In [10]:
BRdata

,Model,Mediator,$m_{med}$,$m_{chi}$,$g_{q}$,$\sin\alpha$,$g_{\chi}$,$y_{\chi}$,$BR(med>\chi\chi)$
0,2MDM,spin-1,1000,100.0,0.01,NaN,0.1,NaN,0.758182
1,2MDM,spin-1,1000,150.0,0.01,NaN,0.1,NaN,0.743169
2,2MDM,spin-1,1000,200.0,0.01,NaN,0.1,NaN,0.719593
3,2MDM,spin-1,1000,250.0,0.01,NaN,0.1,NaN,0.684051
4,2MDM,spin-1,1000,300.0,0.01,NaN,0.1,NaN,0.630542
...,...,...,...,...,...,...,...,...,...
33350,2MDM,spin-0,2000,750.0,NaN,0.25,NaN,2.0,0.060938
33351,2MDM,spin-0,2000,800.0,NaN,0.25,NaN,2.0,0.046199
33352,2MDM,spin-0,2000,850.0,NaN,0.25,NaN,2.0,0.031740
33353,2MDM,spin-0,2000,900.0,NaN,0.25,NaN,2.0,0.018233


### DMSimp Model

In [11]:
## spin-1 mediator ##

# mediator mass
MY1 = np.arange(1000, 4050, 50)

# quarks masses
MU = 0.00255
MC = 1.27
MT = 172
MD = 0.00504
MS = 0.101
MB = 4.7

# DM mass
# MXd = 65.0

# couplings
gAq = 0.0
gAu11 = gAq
gAu22 = gAq
gAu33 = gAq
gAd11 = gAq
gAd22 = gAq
gAd33 = gAq

gVq = 0.25
gVu11 = gVq
gVu22 = gVq
gVu33 = gVq
gVd11 = gVq
gVd22 = gVq
gVd33 = gVq

gAXd = 0.0
gVXd = 1.0

aEWM1 = 127.9
gVh = 0.0

Gf = 0.0000116637

aEW = 1/aEWM1
ee = 2*np.sqrt(aEW)*np.sqrt(np.pi)

# SM bosons masses
MZ = 91.1876
MW = np.sqrt(MZ**2/2. + np.sqrt(MZ**4/4. - (aEW*np.pi*MZ**2)/(Gf*np.sqrt(2))))

sw = np.sqrt(1 - MW**2/MZ**2)
vev = (2*MW*sw)/ee



In [12]:
for mmed in MY1:
    for mx in np.arange(100., mmed/2, 50):
        Y1bb = (mmed**2*(12*gAd33**2*mmed**2 + 12*gVd33**2*mmed**2))/(48.*np.pi*abs(mmed)**3)
        Y1cc = (mmed**2*(12*gAu22**2*mmed**2 + 12*gVu22**2*mmed**2))/(48.*np.pi*abs(mmed)**3)
        Y1dd = (mmed**2*(12*gAd11**2*mmed**2 + 12*gVd11**2*mmed**2))/(48.*np.pi*abs(mmed)**3)
        Y1ss = (mmed**2*(12*gAd22**2*mmed**2 + 12*gVd22**2*mmed**2))/(48.*np.pi*abs(mmed)**3)
        Y1tt = ((-48*gAu33**2*MT**2 + 24*gVu33**2*MT**2 + 12*gAu33**2*mmed**2 + 12*gVu33**2*mmed**2)*np.sqrt(-4*MT**2*mmed**2 + mmed**4))/(48.*np.pi*abs(mmed)**3)
        Y1uu = (mmed**2*(12*gAu11**2*mmed**2 + 12*gVu11**2*mmed**2))/(48.*np.pi*abs(mmed)**3)
        Y1xdxd = ((-16*gAXd**2*mx**2 + 8*gVXd**2*mx**2 + 4*gAXd**2*mmed**2 + 4*gVXd**2*mmed**2)*np.sqrt(-4*mx**2*mmed**2 + mmed**4))/(48.*np.pi*abs(mmed)**3)
        
        # Total width
        
        GammaY1 = Y1bb + Y1cc + Y1dd + Y1ss + Y1tt + Y1uu + Y1xdxd
        
        # BR(y1 to xd xd~)
        
        BR_Y1XdXd = Y1xdxd / GammaY1
        
        
        BRdata.loc[len(BRdata)] = ['DMsimp', 'spin-1', mmed, mx, gVq, np.nan, gVXd, np.nan, BR_Y1XdXd]

In [13]:
BRdata

,Model,Mediator,$m_{med}$,$m_{chi}$,$g_{q}$,$\sin\alpha$,$g_{\chi}$,$y_{\chi}$,$BR(med>\chi\chi)$
0,2MDM,spin-1,1000,100.0,0.01,NaN,0.1,NaN,0.758182
1,2MDM,spin-1,1000,150.0,0.01,NaN,0.1,NaN,0.743169
2,2MDM,spin-1,1000,200.0,0.01,NaN,0.1,NaN,0.719593
3,2MDM,spin-1,1000,250.0,0.01,NaN,0.1,NaN,0.684051
4,2MDM,spin-1,1000,300.0,0.01,NaN,0.1,NaN,0.630542
...,...,...,...,...,...,...,...,...,...
34768,DMsimp,spin-1,4000,1750.0,0.25,NaN,1.0,NaN,0.373068
34769,DMsimp,spin-1,4000,1800.0,0.25,NaN,1.0,NaN,0.352491
34770,DMsimp,spin-1,4000,1850.0,0.25,NaN,1.0,NaN,0.325347
34771,DMsimp,spin-1,4000,1900.0,0.25,NaN,1.0,NaN,0.287142


In [14]:
BRdata.to_pickle('../data/BRdata-models.pcl')

In [16]:
mx = 100
mz = 1000
gq = 0.25
gx = 1.5
BRdata = BRdata[(BRdata['$m_{med}$'] == mz) & (BRdata['$m_{chi}$'] == mx)]
brSimp = BRdata['$BR(med>\chi\chi)$'][(BRdata['Model'] == 'DMsimp')]
br2mdm = BRdata['$BR(med>\chi\chi)$'][(BRdata['Model'] == '2MDM') & (BRdata['Mediator'] == 'spin-1') & (BRdata['$g_{q}$'] == gq) & (BRdata['$g_{\chi}$'] == gx)]

In [17]:
brSimp

33355    0.470664
Name: $BR(med>\chi\chi)$, dtype: float64

In [18]:
br2mdm

25524    0.530235
Name: $BR(med>\chi\chi)$, dtype: float64

In [20]:
float(brSimp)/float(br2mdm)

0.8876514355963231